In [ ]:
#PREPROCESSING

In [2]:
import os
from PIL import Image
import numpy as np

def preprocess_images(input_directory, output_directory, target_size=(224, 224), to_grayscale=True):
    image_count = 0

    for root, dirs, files in os.walk(input_directory):
        for filename in files:
            file_path = os.path.join(root, filename)

            try:
                image = Image.open(file_path)

                image = image.resize(target_size)

                if to_grayscale:
                    image = image.convert('L')

                image_array = np.array(image) / 255.0

                relative_path = os.path.relpath(root, input_directory)
                output_folder = os.path.join(output_directory, relative_path)
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                output_path = os.path.join(output_folder, filename)
                Image.fromarray((image_array * 255).astype(np.uint8)).save(output_path)  # Save as uint8

                image_count += 1

                print(f"Processed and saved: {filename} to {output_path}")

            except Exception as e:
                print(f"Error processing image {file_path}: {e}")

    print(f"Total number of images processed: {image_count}")

# Example usage
input_directory = r'C:\Users\Asus\Desktop\archive(3)'
output_directory = r'"C:\Users\Asus\Desktop\Pre-processed'

preprocess_images(input_directory, output_directory)


Processed and saved: 220.jpg to C:\Users\Asus\Desktop\Pre\test\altar\220.jpg
Processed and saved: 221.jpg to C:\Users\Asus\Desktop\Pre\test\altar\221.jpg
Processed and saved: 222.jpg to C:\Users\Asus\Desktop\Pre\test\altar\222.jpg
Processed and saved: 223.jpg to C:\Users\Asus\Desktop\Pre\test\altar\223.jpg
Processed and saved: 224.jpg to C:\Users\Asus\Desktop\Pre\test\altar\224.jpg
Processed and saved: 225.jpg to C:\Users\Asus\Desktop\Pre\test\altar\225.jpg
Processed and saved: 226.jpg to C:\Users\Asus\Desktop\Pre\test\altar\226.jpg
Processed and saved: 227.jpg to C:\Users\Asus\Desktop\Pre\test\altar\227.jpg
Processed and saved: 228.jpg to C:\Users\Asus\Desktop\Pre\test\altar\228.jpg
Processed and saved: 229.jpg to C:\Users\Asus\Desktop\Pre\test\altar\229.jpg
Processed and saved: 230.jpg to C:\Users\Asus\Desktop\Pre\test\altar\230.jpg
Processed and saved: 231.jpg to C:\Users\Asus\Desktop\Pre\test\altar\231.jpg
Processed and saved: 232.jpg to C:\Users\Asus\Desktop\Pre\test\altar\232.jpg

In [ ]:
#BEFORE PRUNING

In [2]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import mobilenet_v3_small
from torch.utils.tensorboard import SummaryWriter

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data paths
preprocessed_data_dir = r'C:\Users\Asus\Desktop\Pre-processed'
train_dir = os.path.join(preprocessed_data_dir, 'train')
val_dir = os.path.join(preprocessed_data_dir, 'test')

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Define transformations
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for ImageNet
])

# Load datasets
train_dataset = ImageFolder(train_dir, transform=transform)
val_dataset = ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Load MobileNetV3
model = mobilenet_v3_small(pretrained=True)

# Replace the classifier layer (no Dropout)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(train_dataset.classes))

# Save the pre-pruned model state
torch.save(model.state_dict(), 'mobilenetv3_pre_no_regularization.pth')

# Move model to the device
model.to(device)

# Define loss function and optimizer (no regularization)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # No weight decay

# Train the model
num_epochs = 30
writer = SummaryWriter()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    writer.add_scalar('Loss/train', avg_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)

    # Validate the model
    model.eval()
    correct_val = 0
    total_val = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_accuracy = 100 * correct_val / total_val
    writer.add_scalar('Accuracy/val', val_accuracy, epoch)

    # Print results for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {val_accuracy:.2f}%')

# Save the trained (final) model
torch.save(model.state_dict(), 'mobilenetv3_final_no_regularization.pth')

writer.close()


Epoch [1/30], Loss: 0.4136, Train Accuracy: 86.70%, Test Accuracy: 82.83%
Epoch [2/30], Loss: 0.2023, Train Accuracy: 93.45%, Test Accuracy: 87.82%
Epoch [3/30], Loss: 0.1603, Train Accuracy: 94.84%, Test Accuracy: 88.96%
Epoch [4/30], Loss: 0.1180, Train Accuracy: 96.16%, Test Accuracy: 79.63%
Epoch [5/30], Loss: 0.1031, Train Accuracy: 96.54%, Test Accuracy: 89.25%
Epoch [6/30], Loss: 0.0846, Train Accuracy: 97.15%, Test Accuracy: 89.32%
Epoch [7/30], Loss: 0.0837, Train Accuracy: 97.41%, Test Accuracy: 88.89%
Epoch [8/30], Loss: 0.0611, Train Accuracy: 97.99%, Test Accuracy: 89.89%
Epoch [9/30], Loss: 0.0636, Train Accuracy: 97.96%, Test Accuracy: 90.31%
Epoch [10/30], Loss: 0.0649, Train Accuracy: 98.09%, Test Accuracy: 87.75%
Epoch [11/30], Loss: 0.0708, Train Accuracy: 97.86%, Test Accuracy: 90.81%
Epoch [12/30], Loss: 0.0480, Train Accuracy: 98.54%, Test Accuracy: 88.25%
Epoch [13/30], Loss: 0.0636, Train Accuracy: 98.02%, Test Accuracy: 90.88%
Epoch [14/30], Loss: 0.0480, Train

In [ ]:
#FINAL CODE

In [12]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision.models import mobilenet_v3_small
from torch.utils.tensorboard import SummaryWriter
from torch.nn.utils import prune

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data paths
preprocessed_data_dir = r'C:\Users\Asus\Desktop\Pre-processed'
train_dir = os.path.join(preprocessed_data_dir, 'train')
test_dir = os.path.join(preprocessed_data_dir, 'test')

# Image size and batch size
img_size = (224, 224)
batch_size = 32

# Define transformations
transform = transforms.Compose([
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalization for ImageNet
])

# Load datasets
train_dataset = ImageFolder(train_dir, transform=transform)
test_dataset = ImageFolder(test_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load MobileNetV3
model = mobilenet_v3_small(pretrained=True)

# Replace the classifier layer (no Dropout)
model.classifier[3] = nn.Linear(model.classifier[3].in_features, len(train_dataset.classes))

# Save the pre-pruned model state
torch.save(model.state_dict(), 'mobilenetv3_pre_no_regularization.pth')

# Move model to the device
model.to(device)

# Define loss function and optimizer (no regularization)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0)  # No weight decay

# Define learning rate scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Define a function to apply pruning to more layers
def apply_pruning(model):
    # Apply pruning to Conv2d and Linear layers
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv2d) or isinstance(module, nn.Linear):
            parameters_to_prune = (
                (module, 'weight'),
            )
            prune.global_unstructured(
                parameters_to_prune,
                pruning_method=prune.L1Unstructured,
                amount=0.4,
            )

# Function to remove pruning masks and retain only non-zero weights
def remove_pruning(model):
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) or isinstance(module, nn.Conv2d):
            if hasattr(module, 'weight_orig'):  # Check if the layer is pruned
                prune.remove(module, 'weight')

# Function to test the model and return accuracy
def test_model(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Train the model
num_epochs = 30
writer = SummaryWriter()

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    avg_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train
    writer.add_scalar('Loss/train', avg_loss, epoch)
    writer.add_scalar('Accuracy/train', train_accuracy, epoch)

    # Test the model
    test_accuracy = test_model(model, test_loader)
    writer.add_scalar('Accuracy/test', test_accuracy, epoch)

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Test Accuracy: {test_accuracy:.2f}%, Learning Rate: {scheduler.get_last_lr()[0]:.6f}')

    # Step the scheduler at the end of each epoch
    scheduler.step()

# Save the model before pruning
torch.save(model.state_dict(), 'beforePRUNING_no_regularization.pth')
before_pruning_train_accuracy = test_model(model, train_loader)
before_pruning_test_accuracy = test_model(model, test_loader)

# Apply pruning after training
apply_pruning(model)

# Remove pruning masks (to permanently eliminate zero weights)
remove_pruning(model)

# Save the pruned model (only non-zero weights)
sparse_model_path = 'afterPRUNING_sparse_no_regularization.pth'
sparse_state_dict = {}
for name, param in model.state_dict().items():
    # Retain only non-zero weights
    sparse_state_dict[name] = param[param != 0] if param.dim() > 1 else param

# Save the pruned model in a more compact format
torch.save(sparse_state_dict, sparse_model_path)

# Calculate accuracies after pruning
after_pruning_train_accuracy = test_model(model, train_loader)
after_pruning_test_accuracy = test_model(model, test_loader)

# Log the model sizes and accuracies
pre_pruning_size = os.path.getsize('mobilenetv3_pre_no_regularization.pth') / 1024  # KB
post_pruning_size = os.path.getsize(sparse_model_path) / 1024  # KB

print(f"Model Size Before Pruning: {pre_pruning_size:.2f} KB")
print(f"Model Size After Pruning: {post_pruning_size:.2f} KB")
print(f"Train Accuracy Before Pruning: {before_pruning_train_accuracy:.2f}%")
print(f"Test Accuracy Before Pruning: {before_pruning_test_accuracy:.2f}%")
print(f"Train Accuracy After Pruning: {after_pruning_train_accuracy:.2f}%")
print(f"Test Accuracy After Pruning: {after_pruning_test_accuracy:.2f}%")

writer.close()


Epoch [1/30], Loss: 0.4202, Train Accuracy: 86.29%, Test Accuracy: 83.62%, Learning Rate: 0.001000
Epoch [2/30], Loss: 0.2020, Train Accuracy: 93.44%, Test Accuracy: 87.04%, Learning Rate: 0.001000
Epoch [3/30], Loss: 0.1481, Train Accuracy: 95.16%, Test Accuracy: 90.46%, Learning Rate: 0.001000
Epoch [4/30], Loss: 0.1265, Train Accuracy: 95.67%, Test Accuracy: 85.61%, Learning Rate: 0.001000
Epoch [5/30], Loss: 0.0967, Train Accuracy: 96.73%, Test Accuracy: 88.82%, Learning Rate: 0.001000
Epoch [6/30], Loss: 0.0838, Train Accuracy: 97.42%, Test Accuracy: 91.17%, Learning Rate: 0.001000
Epoch [7/30], Loss: 0.0697, Train Accuracy: 97.77%, Test Accuracy: 89.74%, Learning Rate: 0.001000
Epoch [8/30], Loss: 0.0734, Train Accuracy: 97.54%, Test Accuracy: 86.97%, Learning Rate: 0.001000
Epoch [9/30], Loss: 0.0861, Train Accuracy: 97.29%, Test Accuracy: 85.83%, Learning Rate: 0.001000
Epoch [10/30], Loss: 0.0662, Train Accuracy: 97.72%, Test Accuracy: 87.82%, Learning Rate: 0.001000
Epoch [11

In [ ]:
#TESTING THE MODEL

In [6]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_small
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data directory and transform
preprocessed_data_dir = r'C:\Users\Asus\Desktop\Pre-processed'
train_dir = os.path.join(preprocessed_data_dir, 'train')

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset to get class names
train_dataset = ImageFolder(train_dir, transform=transform)

# Load the model
def load_model(model_path, num_classes):
    model = mobilenet_v3_small(pretrained=False)  # Initialize model structure
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)  # Update classifier for your classes
    model.load_state_dict(torch.load(model_path))  # Load your trained weights
    model.to(device)  # Move model to the device
    model.eval()  # Set model to evaluation mode
    return model

# Function to preprocess and predict on a single image
def predict_image(image_path, model, transform):
    # Open image
    image = Image.open(image_path).convert("RGB")
    
    # Apply transformations
    image = transform(image)
    
    # Add batch dimension
    image = image.unsqueeze(0).to(device)  # Shape [1, 3, 224, 224]
    
    # Forward pass
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    # Map the predicted index to the class name
    class_name = train_dataset.classes[predicted.item()]
    return class_name

# Paths to model and test image
model_path = 'afterPRUNING_no_mask.pth'  # Path to the desired model file
test_image_path = r"C:\Users\Asus\Desktop\archive(3)\test_before\apse\bell_tower\1235.jpg"

# Load model and make a prediction
model = load_model(model_path, num_classes=len(train_dataset.classes))
predicted_class = predict_image(test_image_path, model, transform)

print(f"Predicted class: {predicted_class}")


C:\Users\Asus\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Asus\AppData\Local\Temp\ipykernel_25484\1048173841.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializat

Predicted class: bell_tower


In [6]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_small
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data directory and transform
preprocessed_data_dir = r'C:\Users\Asus\Desktop\Pre-processed'
train_dir = os.path.join(preprocessed_data_dir, 'train')

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset to get class names
train_dataset = ImageFolder(train_dir, transform=transform)

# Load the model
def load_model(model_path, num_classes):
    model = mobilenet_v3_small(pretrained=False)  # Initialize model structure
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)  # Update classifier for your classes
    model.load_state_dict(torch.load(model_path))  # Load your trained weights
    model.to(device)  # Move model to the device
    model.eval()  # Set model to evaluation mode
    return model

# Function to preprocess and predict on a single image
def predict_image(image_path, model, transform):
    # Open image
    image = Image.open(image_path).convert("RGB")
    
    # Apply transformations
    image = transform(image)
    
    # Add batch dimension
    image = image.unsqueeze(0).to(device)  # Shape [1, 3, 224, 224]
    
    # Forward pass
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    # Map the predicted index to the class name
    class_name = train_dataset.classes[predicted.item()]
    return class_name

# Paths to model and test image
model_path = 'afterPRUNING_no_mask.pth'  # Path to the desired model file
test_image_path = r"C:\Users\Asus\Desktop\archive(3)\test_before\column\360.jpg"

# Load model and make a prediction
model = load_model(model_path, num_classes=len(train_dataset.classes))
predicted_class = predict_image(test_image_path, model, transform)

print(f"Predicted class: {predicted_class}")


C:\Users\Asus\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Asus\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
C:\Users\Asus\AppData\Local\Temp\ipykernel_25728\560929638.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped t

Predicted class: column


In [2]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from PIL import Image
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v3_small
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Data directory and transform
preprocessed_data_dir = r'C:\Users\Asus\Desktop\Pre-processed'
train_dir = os.path.join(preprocessed_data_dir, 'train')

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load dataset to get class names
train_dataset = ImageFolder(train_dir, transform=transform)

# Load the model
def load_model(model_path, num_classes):
    model = mobilenet_v3_small(pretrained=False)  # Initialize model structure
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)  # Update classifier for your classes
    model.load_state_dict(torch.load(model_path))  # Load your trained weights
    model.to(device)  # Move model to the device
    model.eval()  # Set model to evaluation mode
    return model

# Function to preprocess and predict on a single image
def predict_image(image_path, model, transform):
    # Open image
    image = Image.open(image_path).convert("RGB")
    
    # Apply transformations
    image = transform(image)
    
    # Add batch dimension
    image = image.unsqueeze(0).to(device)  # Shape [1, 3, 224, 224]
    
    # Forward pass
    with torch.no_grad():
        output = model(image)
        _, predicted = torch.max(output, 1)
    
    # Map the predicted index to the class name
    class_name = train_dataset.classes[predicted.item()]
    return class_name

# Paths to model and test image
model_path = 'afterPRUNING_no_mask.pth'  # Path to the desired model file
test_image_path = r"C:\Users\Asus\Desktop\archive(3)\test_before\gargoyle\994.jpg"

# Load model and make a prediction
model = load_model(model_path, num_classes=len(train_dataset.classes))
predicted_class = predict_image(test_image_path, model, transform)

print(f"Predicted class: {predicted_class}")


Predicted class: gargoyle


C:\Users\Asus\AppData\Local\Temp\ipykernel_23228\3889065494.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))  # Load your t